### Importing libraries

In [1]:
import os
import pandas as pd
from pathlib import Path
from datetime import date
from provenance_data_extract_package import prov_packages

### Execution

In [2]:
package = prov_packages()

##### Select the Testset and the comparator bucket

In [5]:
Testset_BUCKET_NAME = package.select_bucket(description='Testset')
Comparator_BUCKET_NAME = package.select_bucket(description='Comparator')

Dropdown(description='Testset:', index=21, options=('290244732740-eu-west-1-s3-logs', '290244732740-mnist', '2…

Dropdown(description='Comparator:', index=21, options=('290244732740-eu-west-1-s3-logs', '290244732740-mnist',…

##### Select the testruns for Testset and comparator

In [6]:
testset = package.response_sub_bucket(bucket_name=Testset_BUCKET_NAME.value,title="Testset")
comparator = package.response_sub_bucket(bucket_name=Comparator_BUCKET_NAME.value,title="Comparator")

Dropdown(description='Testset', layout=Layout(width='300px'), options=('06June2024-uat-2.2ktesting-manualindex…

Dropdown(description='Comparator', layout=Layout(width='300px'), options=('06June2024-uat-2.2ktesting-manualin…

Testset: 15thOct2024-retry-fixes-test-run/


In [7]:
testset_prefix_name = f"{testset.value}intermediate-responses/"
comparator_prefix_name = f"{comparator.value}intermediate-responses/"
testset_files = package.get_filenames(BUCKET_NAME=Testset_BUCKET_NAME.value,prefix_name=testset_prefix_name)
comparator_files = package.get_filenames(BUCKET_NAME=Comparator_BUCKET_NAME.value,prefix_name=comparator_prefix_name)

testset_files = [i for i in testset_files if i.split('/')[-2] not in ["field_extraction","format_conversion","text_to_attributes"]]
comparator_files = [i for i in comparator_files if i.split('/')[-2] not in ["field_extraction","format_conversion","text_to_attributes"]]

testset_services = package.services_data(testset_files,BUCKET_NAME=Testset_BUCKET_NAME.value)
comparator_services = package.services_data(comparator_files,BUCKET_NAME=Comparator_BUCKET_NAME.value)
services_list = ["termite","xpath_union","entity_tag","entity_subheading","label_and_weight","streaming_relevancy"]

services_used_counts = {}
for key in services_list:
        services_used_counts[key] = {f"Comparator_{comparator_files[0].split('/')[0]}-{comparator_files[0].split('/')[2]}":["No diff"] if not list(package.match_pattern(comparator_services[key])-package.match_pattern(testset_services[key])) else list(package.match_pattern(comparator_services[key])-package.match_pattern(testset_services[key])),
                          f"Testset_{testset_files[0].split('/')[0]}-{testset_files[0].split('/')[2]}": ["No diff"] if not list(package.match_pattern(testset_services[key])-package.match_pattern(comparator_services[key])) else list(package.match_pattern(testset_services[key])-package.match_pattern(comparator_services[key]))}


dict_services = {}
for service in services_used_counts.keys():
    dict_services[service] = pd.DataFrame(services_used_counts[service])

### Exporting summary report

In [9]:
path = Path("Summary_Report")
path.mkdir(parents=True,exist_ok=True)
comp_test = str(testset_files[0].split('/')[-3])+"--"+str(comparator_files[0].split('/')[-3])
file_name = f"ProvSummary_{Testset_BUCKET_NAME.value}__{Comparator_BUCKET_NAME.value}_{comp_test}_{date.today()}"
writers = pd.ExcelWriter(f"{path}/{file_name}.xlsx",engine='openpyxl')
for sheet in dict_services.keys():
    dict_services[sheet].to_excel(writers,sheet_name=sheet,index=False)
writers.close()